In [1]:
import pandas as pd
import numpy as np
import os
import re
import string
import gensim
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train = pd.read_csv('data\\train.csv').fillna(' ')
test = pd.read_csv('data\\test.csv').fillna(' ')

In [3]:
from nlp_pipeline import *

In [4]:
N_DIM = 300
# pretrained = "data\\GoogleNews-vectors-negative300.bin.gz"
pretrained = "data\\crawl-300d-2M.vec"

In [5]:
def get_coefs(row):
    row = row.strip().split()
    # can't use row[0], row[1:] split because 840B contains multi-part words 
    word, arr = " ".join(row[:-N_DIMS]), row[-N_DIMS:]
    return word, np.asarray(arr, dtype='float32')

def get_word_vectors(pretrained):
    return dict(get_coefs(row) for row in open(pretrained, encoding="utf-8"))

In [6]:
# w2v = gensim.models.KeyedVectors.load_word2vec_format(pretrained, binary=True)   
fasttext = get_word_vectors(pretrained)

In [7]:
input_column = 'comment_text'
class_labels = [column for column in train.columns[2:8]]
feature_funcs = [lengths, asterixes, uppercase_count]
transforms = [tokenize]
logreg = LogisticRegression(solver='sag')
logreg.name = "Logistic regression newton"
models = [logreg]

In [8]:
pipeline = NlpPipeline(train, test, input_column, class_labels, feature_funcs, transforms, models, word_vectors=fasttext)

In [9]:
pipeline.engineer_features()

Engineering features


In [10]:
pipeline.apply_transforms()

Applying transforms


In [12]:
pipeline.create_embeddings()

Creating embeddings


In [13]:
logreg.name = "Logistic regression sag"

In [ ]:
pipeline.cross_val()

Cross-validating
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)
Cross-validating toxic


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
pipeline.fit_predict()

In [ ]:
pipeline.create_submission()

In [ ]:
pipeline.cv_scores

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('data\\train.csv').fillna(' ')
test = pd.read_csv('data\\test.csv').fillna(' ')

In [ ]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=5000)

In [ ]:
char_vectorizer.fit(all_text)

In [ ]:
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [ ]:
pipeline.engineer_features()

In [ ]:
pipeline.train_features = pipeline.normalize(pipeline.train_features)
pipeline.test_features = pipeline.normalize(pipeline.test_features)

In [ ]:
pipeline.train_features.shape

In [ ]:
train_char_features[:10]

In [ ]:
pipeline.train_features = train_char_features

In [ ]:
pipeline.test_features = test_char_features

In [ ]:
type(pipeline.test_features)

In [ ]:
from scipy import sparse

In [ ]:
sparse(np.array([1,2,3]))

In [ ]:
from scipy import hstack
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

In [ ]:
test = hstack((pipeline.train_features, train_char_features))

In [ ]:
type(pipeline.train_features.tocsr())

In [ ]:
train_features = sparse.hstack([sparse.csr_matrix(pipeline.train_features), train_char_features]).tocsr()

In [ ]:
pipeline.train_features = train_char_features

In [ ]:
pipeline.test_features = test_char_features

In [ ]:
pipeline

In [ ]:
pipeline.cross_val()

In [ ]:
pipeline.cross_val()

In [ ]:
pipeline.cv_scores

In [ ]:
pipeline.fit_predict()

In [ ]:
pipeline.create_submission()

In [15]:
print("OK")

OK


In [19]:
pipeline.models

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='sag', tol=0.0001,
           verbose=0, warm_start=False)]